In [2]:
import numpy as np
from collections import Counter

In [5]:
def entropy(x):
    hist=np.bincount(x)
    print(hist)
    prob_list=hist/len(x)
    return np.sum([-p*np.log2(p) for p in prob_list])

# entropy defines randomness 

a=[1,2,1,1,3,4,5,7]
res=entropy(a)
print(res)


[0 3 1 1 1 1 0 1]
nan


/var/folders/cc/9wx87yxd1l1g7q88nc0g_tg00000gn/T/ipykernel_1544/285912300.py:5: RuntimeWarning: divide by zero encountered in log2
  return np.sum([-p*np.log2(p) for p in prob_list])
/var/folders/cc/9wx87yxd1l1g7q88nc0g_tg00000gn/T/ipykernel_1544/285912300.py:5: RuntimeWarning: invalid value encountered in scalar multiply
  return np.sum([-p*np.log2(p) for p in prob_list])


In [14]:
a=[1,2,3,4,5]
a=np.array(a)
print(a.shape)
# column vector
res=np.argwhere(a>=3)
print(res.shape,res)
nres=res.flatten()
print(nres.shape,nres)


# returns the indices with given condition 

(5,)
(3, 1) [[2]
 [3]
 [4]]
(3,) [2 3 4]


In [62]:
def entropy(x):
    hist=np.bincount(x)
    host_prob_list=hist/len(x)
    return -np.sum([p*np.log2(p) for p in host_prob_list])

class Node():
    def __init__(self,feature=None,threshold=None,left=None,right=None,*,value=None):
        self.feature=feature
        self.threshold=threshold
        self.left=left
        self.right=right
        self.value=value

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree():
    def __init__(self,max_depth=100,min_sample_split=2,n_feats=None):
        self.max_depth=max_depth
        self.min_sample_split=min_sample_split
        self.n_feats=n_feats
        self.root=None

    def fit(self,x,y):
        self.n_feats=x.shape[1] if not self.n_feats else min(x.shape[1],self.n_feats)
        self.root=self._grow_tree(x,y)
        

    def _grow_tree(self,x,y,depth=0):
        n_samples,n_features=x.shape
        n_labels=len(np.unique(y))

        if(n_labels==1 or depth==self.max_depth or n_samples<=self.min_sample_split):
            leaf_value=self._most_common(y)
            return Node(value=leaf_value)

        feature_idx=np.random.choice(n_features,self.n_feats,replace=False)
        best_feature_idx,best_threshold=self._best_split_index(x,y,feature_idx)
        left_idx,right_idx=self._split_idx(x[:,best_feature_idx],best_threshold)
        if len(left_idx) == 0 or len(right_idx) == 0:
            leaf_value = self._most_common(y)
            return Node(value=leaf_value)

        left=self._grow_tree(x[left_idx,: ],y[left_idx],depth+1)
        right=self._grow_tree(x[right_idx,: ],y[right_idx],depth+1)
        return Node(best_feature_idx,best_threshold,left,right)

    

    def _most_common(self,x):
        res= Counter(x).most_common(1)
        return res[0][0]


    def _best_split_index(self,x,y,feature_idx):
        bestgain=-1
        best_idx,best_threshold=None,None
        for fidx in feature_idx:
            x_col=x[:,fidx]
            thresholds=np.unique(x_col)
            for thresh in thresholds:
                gain=self._information_gain(x_col,y,thresh)
                if gain>bestgain:
                    bestgain=gain
                    best_idx=fidx
                    best_threshold=thresh
        return best_idx,best_threshold


    def _information_gain(self,x,y,threshold):
        parent_entropy=entropy(y)
        left_idx,right_idx=self._split_idx(x,threshold)
        if len(left_idx) == 0 or len(right_idx) == 0:
            return 0
        llen,rlen=len(left_idx),len(right_idx)
        n=len(y)
        child_entropy=(llen/n)*entropy(y[left_idx]) + (rlen/n)*entropy(y[right_idx])
        return parent_entropy-child_entropy


    def _split_idx(self,x,val):
        left=np.argwhere(x<=val).flatten()
        right=np.argwhere(x>val).flatten()
        return left,right

    def predict(self,x_test):
        return np.array([self._traverse_tree(x,self.root) for x in x_test])

    def _traverse_tree(self,x,node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature]<=node.threshold:
            return self._traverse_tree(x,node.left)
        return self._traverse_tree(x,node.right)


In [63]:
import numpy as np
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split


In [64]:
data=datasets.load_breast_cancer()
x,y=data.data,data.target

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1234)


In [65]:
clf=DecisionTree(max_depth=10)
clf.fit(x_train,y_train)

/var/folders/cc/9wx87yxd1l1g7q88nc0g_tg00000gn/T/ipykernel_1544/2427416723.py:4: RuntimeWarning: divide by zero encountered in log2
  return -np.sum([p*np.log2(p) for p in host_prob_list])
/var/folders/cc/9wx87yxd1l1g7q88nc0g_tg00000gn/T/ipykernel_1544/2427416723.py:4: RuntimeWarning: invalid value encountered in scalar multiply
  return -np.sum([p*np.log2(p) for p in host_prob_list])


In [66]:
clf.predict(x_test)

array([1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0])

In [67]:
acc=np.sum(clf.predict(x_test)==y_test)/len(y_test)

In [68]:
acc

np.float64(0.9210526315789473)